In [1]:
# Importing the libraries 

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

# Ignore harmless warnings 

import warnings 
warnings.filterwarnings("ignore")

# Set to display all the columns in dataset

pd.set_option("display.max_columns", 50)

# Import psql to run queries 

import pandasql as psql

In [2]:
# Load the Universal bank data

bankdata = pd.read_csv(r"D:\00 Datasets\Others\Data-06\Universalbank.csv", header=0) 
bankdata.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [3]:
# Delete the columns which are not influencing the target variable

del bankdata['ID']
del bankdata['ZIP Code']

In [4]:
# cols1 is variables - crating a dummy variables & cols2 variables - MinMaxScalar function

cols1 = ['Family', 'Education']
cols2 = ['Age', 'Experience', 'Income', 'CCAvg', 'Mortgage']

In [5]:
# Create dummy variable for all range values

bankdata = pd.get_dummies(bankdata, columns=cols1)
bankdata.head().T

,0,1,2,3,4
Age,25.0,45.0,39.0,35.0,35.0
Experience,1.0,19.0,15.0,9.0,8.0
Income,49.0,34.0,11.0,100.0,45.0
CCAvg,1.6,1.5,1.0,2.7,1.0
Mortgage,0.0,0.0,0.0,0.0,0.0
Personal Loan,0.0,0.0,0.0,0.0,0.0
Securities Account,1.0,1.0,0.0,0.0,0.0
CD Account,0.0,0.0,0.0,0.0,0.0
Online,0.0,0.0,0.0,0.0,0.0
CreditCard,0.0,0.0,0.0,0.0,1.0


In [6]:
# Identify the dependent and Target variables

IndepVar = []
for col in bankdata.columns:
    if col != 'CreditCard':
        IndepVar.append(col)

TargetVar = 'CreditCard'

x = bankdata[IndepVar]
y = bankdata[TargetVar]

In [7]:
# Splitting the dataset into train and test 

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, stratify=y, random_state=142)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3500, 16), (1500, 16), (3500,), (1500,))

In [8]:
# Scaling the features by using MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

mmscaler = MinMaxScaler(feature_range=(0, 1))

x_train[cols2] = mmscaler.fit_transform(x_train[cols2])
x_train = pd.DataFrame(x_train)

x_test[cols2] = mmscaler.fit_transform(x_test[cols2])
x_test = pd.DataFrame(x_test)

# AdaBoost Classifier 

In [9]:
# Adaboost classifer is used for classification problems

from sklearn.ensemble import AdaBoostClassifier

modelABC = AdaBoostClassifier(base_estimator=None, n_estimators=50, learning_rate=1.0, 
                              algorithm='SAMME.R', random_state=None)

# Fit the model with train data

modelABC.fit(x_train, y_train)

# Predict the model with test data

y_pred = modelABC.predict(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

mx = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(mx), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))

Confusion matrix : 
 [[  79  362]
 [  27 1032]]
Outcome values : 
 79 362 27 1032
Classification report : 
               precision    recall  f1-score   support

           1       0.75      0.18      0.29       441
           0       0.74      0.97      0.84      1059

    accuracy                           0.74      1500
   macro avg       0.74      0.58      0.57      1500
weighted avg       0.74      0.74      0.68      1500

Accuracy : 74.1 %
Precision : 74.5 %
Recall : 17.9 %
F1 Score : 0.289
Balanced Accuracy : 57.7 %
MCC : 0.273
roc_auc_score: 0.577


# Decision Tree as Base Estimator

In [10]:
# Adaboost classifer is used for classification problems with Decision Tree as base estimator

from sklearn.tree import DecisionTreeClassifier

modelDT = DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=3, min_samples_split=2, 
                                 min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, 
                                 random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                 min_impurity_split=None, class_weight=None, ccp_alpha=0.0)

# Create adaboost classifer object

from sklearn.ensemble import AdaBoostClassifier

modelABC1 = AdaBoostClassifier(base_estimator=modelDT, n_estimators=50, learning_rate=1.0,
                               algorithm='SAMME.R', random_state=None)

# Train Adaboost Classifer

modelABC1 = modelABC1.fit(x_train, y_train)

#Predict the response for test dataset

y1_pred = modelABC1.predict(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y1_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

mx = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(mx), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y1_pred), 3))

Confusion matrix : 
 [[109 332]
 [145 914]]
Outcome values : 
 109 332 145 914
Classification report : 
               precision    recall  f1-score   support

           1       0.43      0.25      0.31       441
           0       0.73      0.86      0.79      1059

    accuracy                           0.68      1500
   macro avg       0.58      0.56      0.55      1500
weighted avg       0.64      0.68      0.65      1500

Accuracy : 68.2 %
Precision : 42.9 %
Recall : 24.7 %
F1 Score : 0.314
Balanced Accuracy : 55.5 %
MCC : 0.134
roc_auc_score: 0.555


# SVM as base estimator

In [11]:
# Adaboost classifer is used for classification problems with SVMas base estimator

from sklearn.svm import SVC

modelSVM = SVC(C=1.0, kernel='linear', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=True, 
               tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=- 1, decision_function_shape='ovr', 
               break_ties=False, random_state=None)

# Create adaboost classifer object

modelABC2 = AdaBoostClassifier(base_estimator=modelSVM, n_estimators=50, learning_rate=1.0,
                               algorithm='SAMME', random_state=None)

# Train Adaboost Classifer

modelABC2 = modelABC2.fit(x_train, y_train)

#Predict the response for test dataset

y2_pred = modelABC2.predict(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y2_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

mx = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(mx), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y2_pred), 3))

Confusion matrix : 
 [[   0  441]
 [   0 1059]]
Outcome values : 
 0 441 0 1059
Classification report : 
               precision    recall  f1-score   support

           1       0.00      0.00      0.00       441
           0       0.71      1.00      0.83      1059

    accuracy                           0.71      1500
   macro avg       0.35      0.50      0.41      1500
weighted avg       0.50      0.71      0.58      1500

Accuracy : 70.6 %
Precision : nan %
Recall : 0.0 %
F1 Score : 0.0
Balanced Accuracy : 50.0 %
MCC : nan
roc_auc_score: 0.5


# Random Forest as base estimator

In [12]:
# Adaboost classifer is used for classification problems with RandomForestClassifier as base estimator

from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=None, min_samples_split=2, 
                                 min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                                 max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                                 bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, 
                                 warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

# Create adaboost classifer object

modelABC3 = AdaBoostClassifier(base_estimator=modelRF, n_estimators=50, learning_rate=1.0,
                               algorithm='SAMME', random_state=None)

# Train Adaboost Classifer

modelABC3 = modelABC3.fit(x_train, y_train)

#Predict the response for test dataset

y3_pred = modelABC3.predict(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y3_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

mx = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(mx), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y3_pred), 3))

Confusion matrix : 
 [[105 336]
 [ 78 981]]
Outcome values : 
 105 336 78 981
Classification report : 
               precision    recall  f1-score   support

           1       0.57      0.24      0.34       441
           0       0.74      0.93      0.83      1059

    accuracy                           0.72      1500
   macro avg       0.66      0.58      0.58      1500
weighted avg       0.69      0.72      0.68      1500

Accuracy : 72.4 %
Precision : 57.4 %
Recall : 23.8 %
F1 Score : 0.337
Balanced Accuracy : 58.2 %
MCC : 0.229
roc_auc_score: 0.582


# AdaBoost Classifier with Loan Data

In [16]:
# Load the Loan data

loans = pd.read_csv(r"D:\00 Datasets\Others\Data-01\loan_data.csv", header=0)
loans.head()

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


In [17]:
# Change the name of variable

loans = loans.rename(columns = {'not.fully.paid': 'NFPaid'}, inplace = False)
loans.head()

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,NFPaid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


In [18]:
cat_cols = ['purpose']

loans = pd.get_dummies(loans,columns=cat_cols)

loans=pd.DataFrame(loans)
loans.head()

,credit.policy,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,NFPaid,purpose_all_other,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_major_purchase,purpose_small_business
0,1,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0,0,0,1,0,0,0,0
1,1,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0,0,1,0,0,0,0,0
2,1,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0,0,0,1,0,0,0,0
3,1,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0,0,0,1,0,0,0,0
4,1,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0,0,1,0,0,0,0,0


In [19]:
# Identify the independent and Target (dependent) variables

IndepVar = []
for col in loans.columns:
    if col != 'NFPaid':
        IndepVar.append(col)

TargetVar = 'NFPaid'

x = loans[IndepVar]
y = loans[TargetVar]

In [23]:
# Splitting the dataset into train and test 

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, random_state = 42) 

In [24]:
loans.columns

Index(['credit.policy', 'int.rate', 'installment', 'log.annual.inc', 'dti',
       'fico', 'days.with.cr.line', 'revol.bal', 'revol.util',
       'inq.last.6mths', 'delinq.2yrs', 'pub.rec', 'NFPaid',
       'purpose_all_other', 'purpose_credit_card',
       'purpose_debt_consolidation', 'purpose_educational',
       'purpose_home_improvement', 'purpose_major_purchase',
       'purpose_small_business'],
      dtype='object')

In [25]:
cols1 = ['int.rate', 'installment', 'log.annual.inc', 'dti', 'fico', 'days.with.cr.line', 'revol.bal', 'revol.util']

In [26]:
# Scaling the features by using MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

mmscaler = MinMaxScaler(feature_range=(0, 1))

x_train[cols1] = mmscaler.fit_transform(x_train[cols1])
x_train = pd.DataFrame(x_train)

x_test[cols1] = mmscaler.fit_transform(x_test[cols1])
x_test = pd.DataFrame(x_test)

# AdaBoost Classifier with base estimator as 'none'

In [27]:
# Adaboost classifer is used for classification problems

from sklearn.ensemble import AdaBoostClassifier

modelABC = AdaBoostClassifier(base_estimator=None, n_estimators=50, learning_rate=1.0, 
                              algorithm='SAMME.R', random_state=None)

# Fit the model with train data

modelABC.fit(x_train, y_train)

# Predict the model with test data

y_pred = modelABC.predict(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

mx = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(mx), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))

Confusion matrix : 
 [[  23  443]
 [  34 2374]]
Outcome values : 
 23 443 34 2374
Classification report : 
               precision    recall  f1-score   support

           1       0.40      0.05      0.09       466
           0       0.84      0.99      0.91      2408

    accuracy                           0.83      2874
   macro avg       0.62      0.52      0.50      2874
weighted avg       0.77      0.83      0.78      2874

Accuracy : 83.4 %
Precision : 40.4 %
Recall : 4.9 %
F1 Score : 0.088
Balanced Accuracy : 51.8 %
MCC : 0.093
roc_auc_score: 0.518


# AdaBoost classifier - base estimator is Decision Tree

In [28]:
# Adaboost classifer is used for classification problems with Decision Tree as base estimator

from sklearn.tree import DecisionTreeClassifier

modelDT = DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=None, min_samples_split=2, 
                                 min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, 
                                 random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                 min_impurity_split=None, class_weight=None, ccp_alpha=0.0)

# Create adaboost classifer object

from sklearn.ensemble import AdaBoostClassifier

modelABC1 = AdaBoostClassifier(base_estimator=modelDT, n_estimators=50, learning_rate=1.0,
                               algorithm='SAMME.R', random_state=None)

# Train Adaboost Classifer

modelABC1 = modelABC1.fit(x_train, y_train)

#Predict the response for test dataset

y1_pred = modelABC1.predict(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y1_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

mx = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(mx), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y1_pred), 3))

Confusion matrix : 
 [[ 131  335]
 [ 483 1925]]
Outcome values : 
 131 335 483 1925
Classification report : 
               precision    recall  f1-score   support

           1       0.21      0.28      0.24       466
           0       0.85      0.80      0.82      2408

    accuracy                           0.72      2874
   macro avg       0.53      0.54      0.53      2874
weighted avg       0.75      0.72      0.73      2874

Accuracy : 71.5 %
Precision : 21.3 %
Recall : 28.1 %
F1 Score : 0.243
Balanced Accuracy : 54.0 %
MCC : 0.072
roc_auc_score: 0.54


# AdaBoost classifier - base estimator is SVM

In [29]:
# Adaboost classifer is used for classification problems with SVMas base estimator

from sklearn.svm import SVC

modelSVM = SVC(C=1.0, kernel='linear', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=True, 
               tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=- 1, decision_function_shape='ovr', 
               break_ties=False, random_state=None)

# Create adaboost classifer object

modelABC2 = AdaBoostClassifier(base_estimator=modelSVM, n_estimators=50, learning_rate=1.0,
                               algorithm='SAMME', random_state=None)

# Train Adaboost Classifer

modelABC2 = modelABC2.fit(x_train, y_train)

#Predict the response for test dataset

y2_pred = modelABC2.predict(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y2_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

mx = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(mx), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y2_pred), 3))

Confusion matrix : 
 [[   0  466]
 [   0 2408]]
Outcome values : 
 0 466 0 2408
Classification report : 
               precision    recall  f1-score   support

           1       0.00      0.00      0.00       466
           0       0.84      1.00      0.91      2408

    accuracy                           0.84      2874
   macro avg       0.42      0.50      0.46      2874
weighted avg       0.70      0.84      0.76      2874

Accuracy : 83.8 %
Precision : nan %
Recall : 0.0 %
F1 Score : 0.0
Balanced Accuracy : 50.0 %
MCC : nan
roc_auc_score: 0.5


# AdaBoost classifier - base estimator is Random Forest

In [30]:
# Adaboost classifer is used for classification problems with RandomForestClassifier as base estimator

from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=2, min_samples_split=2, 
                                 min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                                 max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                                 bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, 
                                 warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

# Create adaboost classifer object

modelABC3 = AdaBoostClassifier(base_estimator=modelRF, n_estimators=50, learning_rate=1.0,
                               algorithm='SAMME', random_state=None)

# Train Adaboost Classifer

modelABC3 = modelABC3.fit(x_train, y_train)

#Predict the response for test dataset

y3_pred = modelABC3.predict(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y3_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

mx = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(mx), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y3_pred), 3))

Confusion matrix : 
 [[  45  421]
 [  99 2309]]
Outcome values : 
 45 421 99 2309
Classification report : 
               precision    recall  f1-score   support

           1       0.31      0.10      0.15       466
           0       0.85      0.96      0.90      2408

    accuracy                           0.82      2874
   macro avg       0.58      0.53      0.52      2874
weighted avg       0.76      0.82      0.78      2874

Accuracy : 81.9 %
Precision : 31.2 %
Recall : 9.7 %
F1 Score : 0.148
Balanced Accuracy : 52.8 %
MCC : 0.094
roc_auc_score: 0.528
